In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_sup_region 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
SELECT
    lifnr AS vendor,
    land1 AS country_key,
    lfreg AS supply_region,
    spras AS language_key,
    vtext AS description
FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm422
WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm422)
QUALIFY ROW_NUMBER() OVER (PARTITION BY lifnr, land1, lfreg, spras, vtext ORDER BY proc_date DESC) = 1;
""")